In [ ]:
!pip install mesa

In [ ]:
# 'Model' sirve para definir los atributos a nivel del modelo, maneja los agentes
# 'Agent' es la unidad atómica y puede ser contenido en múltiples instancias en los modelos
from mesa import Agent, Model 

# 'SingleGrid' sirve para forzar a un solo objeto por celda (nuestro objetivo en este "juego")
from mesa.space import SingleGrid

# 'SimultaneousActivation' habilita la opción de activar todos los agentes de manera simultanea.
from mesa.time import SimultaneousActivation

# 'DataCollector' permite obtener el grid completo a cada paso (o generación), útil para visualizar
from mesa.datacollection import DataCollector

# 'matplotlib' lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2 ** 128

# Definimos los siguientes paquetes para manejar valores númericos: 'numpy' & 'pandas'
import numpy as np
import pandas as pd

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time
import datetime
import random
import json

In [ ]:
paleta = np.array(
  [[255, 255, 255],   # blanco
  [139, 115, 85],     # cafe          139,115,85
  [227, 207, 87],     # amarillo      227,207,87
  [83, 134, 139],     # verde         83,134,139
  [207, 207, 87],     # amarillo2     207,207,87
  [187, 207, 87],     # amarillo3     187,207,87
  [167, 207, 87],     # amarillo4     167,207,87
  [147, 207, 87],     # amarillo5     147,207,87
  [127, 207, 87],     # amarillo6     127,207,87
  [83, 94, 79]])      # verde2        83,94,139
#cmap[4] = np.array([0/256, 50/256, 100/256, 1])      # azul

pinta_me = lambda matriz: paleta[matriz.astype(int)]

In [ ]:
class Caja(Agent):
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)

class DropZone(Agent):
  def __init__(self, unique_id, model, NumCajas=0):
    super().__init__(unique_id, model)
    self.NumCajas = NumCajas
    self.color = 2   

class Robot(Agent):
  def __init__(self, unique_id, model, estado=0):
    super().__init__(unique_id, model)
    #self.next_pos = None
    self.estado = estado
    self.color = 3

  def step(self):
    #if self.estado == 0:
    posible_steps_up = []
    posible_steps_sides = []
    posible_steps_down = []
    for ii, pos in enumerate(self.model.grid.get_neighborhood(self.pos, moore=False, include_center=False)):
      cell_content = self.model.grid.get_cell_list_contents([pos])
      if len(cell_content) == 1:
        agente = cell_content[0]
      else:
        agente = None

      #print(ii, agente)

      should_move = True

      if isinstance(agente, DropZone):
        if self.estado == 0:
          continue
        elif self.estado == 1:
          #if agente.NumCajas == 5:
            #continue
          if agente.NumCajas < 5:
            agente.NumCajas += 1
            if agente.NumCajas == 1:
              agente.color = 4
            elif agente.NumCajas == 2:
              agente.color = 5
            elif agente.NumCajas == 3:
              agente.color = 6
            elif agente.NumCajas == 4:
              agente.color = 7
            elif agente.NumCajas == 5:
              agente.color = 8
          self.estado = 0
          self.color = 3
          should_move = False
          #break
      if isinstance(agente, Caja):
        if self.estado == 0:
          self.model.grid.remove_agent(agente)
          self.estado = 1
          self.color = 9
          should_move = False
          break
        elif self.estado == 1:
          continue
      

      if agente is None:
        if self.pos[0] > pos[0]:
          posible_steps_up.append(pos)
        if self.pos[0] < pos[0]:
          posible_steps_down.append(pos)
        if self.pos[0] == pos[0]:
          posible_steps_sides.append(pos)

    #print(posible_steps,self.estado)
      
    if self.estado == 1:
      if should_move:
        if len(posible_steps_up) == 0:
          if len(posible_steps_sides) == 0:
            next_pos = self.pos
          else:
            next_pos = random.choice(posible_steps_sides)
        else:
          next_pos = random.choice(posible_steps_up)
      else:
        next_pos = self.pos
      #if should_move and self.model.grid.is_cell_empty(next_pos):
      #  self.model.grid.move_agent(self, next_pos)
    else:
      if should_move:
        if len(posible_steps_down) == 0:
          if len(posible_steps_sides) == 0:
            next_pos = self.pos
          else:
            next_pos = random.choice(posible_steps_sides) 
        else:
          if len(posible_steps_sides) == 0:
            next_pos = random.choice(posible_steps_down)
          else:
            next_pos = random.choice(random.choice([posible_steps_down, posible_steps_sides]))
      else:
        next_pos = self.pos
      #if should_move and self.model.grid.is_cell_empty(next_pos):
      #  self.model.grid.move_agent(self, next_pos)
    
    #if self.model.grid.out_of_bounds(next_pos):
      #if should_move and self.model.grid.is_cell_empty(next_pos):
      #self.next_pos = self.model.grid.torus_adj(next_pos)
    #else:
      #if should_move and self.model.grid.is_cell_empty(next_pos):
    self.next_pos = next_pos

  #def advance(self):
    if not ((self.next_pos[0] == self.pos[0]) and (self.next_pos[1] == self.pos[1])):
      self.model.grid.move_agent(self, self.next_pos)
   
    """
    x = 1
    y = 1

    if self.estado == 0:
      new_position = (self.pos[0],self.pos[1]+y)
      self.model.grid.move_agent(self, new_position)
    elif self.estado == 1:
      new_position = (self.pos[0],self.pos[1]-y)
      self.model.grid.move_agent(self, new_position)
    """
    #possible_caja = self.get_cell_list_contents(Caja)
    #new_position = self.random.choice(possible_steps)
    #self.model.grid.move_agent(self, new_position)



In [ ]:
class Robot_Cajas(Model):
  def __init__(self, width, height):
    self.num_agents = width * height
    self.grid = SingleGrid(width, height, False)
    self.schedule = SimultaneousActivation(self)

    cont = 0
    #Spawn de "Drop zone"
    for y in range(10):
      dz = DropZone(cont, self)
      self.grid.place_agent(dz, (0, y))
      self.schedule.add(dz)
      cont += 1

    #Spawn de "Robots"
    for i in range(5):
      r = Robot(cont, self)
      self.grid.place_agent(r, (1, i+i))
      self.schedule.add(r)
      cont += 1

    play_zone = [(x, y) for _, x, y in self.grid.coord_iter() if x > 2]
    is_here_a_caja = random.choices(play_zone, k=60)

    #Spawn de "Cajas"
    for content, x, y in self.grid.coord_iter():
      #if (not content) and (x, y not in [0, 1]) and (x, y not in [0, 1]):
      if (x, y) in is_here_a_caja:
          c = Caja(cont, self)
          self.grid.place_agent(c, (x, y))
          #self.schedule.add(c)
          cont += 1
 
    # Aquí definimos el colector de datos para obtener el grid completo.
    self.datacollector = DataCollector(
      model_reporters={"Grid": self.get_grid,
                       "JSON_step": self.read_agents}
    )
    
  def step(self):
    self.datacollector.collect(self)
    self.schedule.step()

  def get_grid(self):
    # Generamos la grid para contener los valores
    grid = np.zeros((self.grid.width, self.grid.height))
    
    for cell in self.grid.coord_iter():
      cell_content, x, y = cell
      if isinstance(cell_content, Caja):
        grid[x][y] = 1
      elif isinstance(cell_content, Robot):
        grid[x][y] = cell_content.color
      elif isinstance(cell_content, DropZone): 
        grid[x][y] = cell_content.color
      else: 
        grid[x][y] = 0     

    return grid
  
  def read_agents(self):

    #Por todas las celdas del grid
    agents_list = []
    for cell in model.grid.coord_iter():
      agent, x, y = cell

      if isinstance(agent, Robot):
        agent_dict = {
          'id': agent.unique_id,
          'kind': 'robot',
          'positionX': agent.pos[0],
          'positionY': 0,
          'positionZ': agent.pos[1],
          'color': agent.color,
          'estado': agent.estado
        }

      elif isinstance(agent, Caja):
        agent_dict = {
          'id': agent.unique_id,
          'kind': 'caja',
          'positionX': agent.pos[0],
          'positionY': 0,
          'positionZ': agent.pos[1]
        }
      
      elif isinstance(agent, DropZone):
        agent_dict = {
          'id': agent.unique_id,
          'kind': 'dropZone',
          'positionX': agent.pos[0],
          'positionY': 0,
          'positionZ': agent.pos[1],
          'color': agent.color,
          'numCajas': agent.NumCajas
        }
      else:
          continue
      agents_list.append(agent_dict)

    return agents_list

In [ ]:
# Definimos el tamaño del Grid
GRID_SIZE = 10

# Definimos el número de generaciones a correr
NUM_GENERATIONS = 100

# Registramos el tiempo de inicio y corremos el modelo
start_time = time.time()
model = Robot_Cajas(GRID_SIZE, GRID_SIZE)

archivo = open("test.json", "w")

for i in range(NUM_GENERATIONS):
  model.step()

  jsn = model.read_agents()

  for json_object in jsn:
    archivo.write(json.dumps(json_object, indent=4))

archivo.close()

# Imprimimos el tiempo que le tomó correr al modelo.
print('Tiempo de ejecución:', str(datetime.timedelta(seconds=(time.time() - start_time))))

Tiempo de ejecución: 0:00:00.204500


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()

In [ ]:
all_grid
ejemplo = {}
for i in range (len(all_grid)):
  ejemplo[i] = all_grid["JSON_step"].iloc[i]

In [ ]:
ejemplo2 = all_grid["JSON_step"]
ejemplo2.to_json("test.json")

In [ ]:
ejemplo2

0     [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
1     [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
2     [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
3     [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
4     [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
                            ...                        
95    [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
96    [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
97    [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
98    [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
99    [{'id': 0, 'kind': 'dropZone', 'positionX': 0,...
Name: JSON_step, Length: 100, dtype: object

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(pinta_me(all_grid.iloc[0][0]))

def animate(i):
  patch.set_data(pinta_me(all_grid.iloc[i][0]))
    
anim = animation.FuncAnimation(fig, animate, frames=NUM_GENERATIONS)

In [ ]:
anim